In [1]:
import numpy as np
import pandas as pd
from django.db.models import Sum
from public_data.models import OcsgeDiff, CouvertureSol
from project.models import Project

In [8]:
project = Project.objects.get(pk=8)
project

<Project: Diagnostic de CC de la Tenarèze>

In [9]:
cities = CommuneDiff.objects.filter(city__in=project.cities.all())
cities

<QuerySet [<CommuneDiff: CommuneDiff object (175)>, <CommuneDiff: CommuneDiff object (174)>, <CommuneDiff: CommuneDiff object (94)>, <CommuneDiff: CommuneDiff object (283)>, <CommuneDiff: CommuneDiff object (132)>, <CommuneDiff: CommuneDiff object (106)>, <CommuneDiff: CommuneDiff object (448)>, <CommuneDiff: CommuneDiff object (391)>, <CommuneDiff: CommuneDiff object (99)>, <CommuneDiff: CommuneDiff object (141)>, <CommuneDiff: CommuneDiff object (207)>, <CommuneDiff: CommuneDiff object (219)>, <CommuneDiff: CommuneDiff object (395)>, <CommuneDiff: CommuneDiff object (79)>, <CommuneDiff: CommuneDiff object (58)>, <CommuneDiff: CommuneDiff object (225)>, <CommuneDiff: CommuneDiff object (191)>, <CommuneDiff: CommuneDiff object (74)>, <CommuneDiff: CommuneDiff object (192)>, <CommuneDiff: CommuneDiff object (39)>, '...(remaining elements truncated)...']>

In [10]:
headers = CouvertureSol.objects.all().values_list("code_prefix", flat=True)
headers

<QuerySet ['CS1', 'CS1.1', 'CS1.1.1', 'CS1.1.1.1', 'CS1.1.1.2', 'CS1.1.2', 'CS1.1.2.1', 'CS1.1.2.2', 'CS1.2', 'CS1.2.1', 'CS1.2.2', 'CS1.2.3', 'CS2', 'CS2.1', 'CS2.1.1', 'CS2.1.1.1', 'CS2.1.1.2', 'CS2.1.1.3', 'CS2.1.2', 'CS2.1.3', '...(remaining elements truncated)...']>

In [11]:
qs = (
    OcsgeDiff.objects.intersect(project.combined_emprise)
    .filter(year_old__gte=project.analyse_start_date, year_new__lte=project.analyse_end_date)
    .values("cs_new", "cs_old")
    .annotate(total=Sum("surface") / 10000)
    .order_by("cs_new", "cs_old")
)
qs

<QuerySet [{'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.1.1', 'total': Decimal('0.77158466000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.1.2', 'total': Decimal('0.85468720000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS1.1.2.1', 'total': Decimal('1.03019264000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.1.1.1', 'total': Decimal('0.08408434000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.1.3', 'total': Decimal('0.07158986000000000000')}, {'cs_new': 'CS1.1.1.1', 'cs_old': 'CS2.2.1', 'total': Decimal('4.9698640900000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.1.1', 'total': Decimal('0.18134972000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.1.2', 'total': Decimal('0.39488423000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS1.1.2.1', 'total': Decimal('1.00606014000000000000')}, {'cs_new': 'CS1.1.1.2', 'cs_old': 'CS2.2.1', 'total': Decimal('2.1550538900000000')}, {'cs_new': 'CS1.1.2.1', 'cs_old': 'CS1.1.1.1', 'total': Decimal('0.4008812700000000

In [12]:
pd.options.display.float_format = '${:,.2f}'.format
df = (
    pd.DataFrame(qs)
    .pivot(index='cs_old', columns='cs_new', values='total')
    .fillna(0)
    
)
df

cs_new,NaN,CS1.1.1.1,CS1.1.1.2,CS1.1.2.1,CS1.1.2.2,CS1.2.2,CS2.1.1.1,CS2.1.2,CS2.1.3,CS2.2.1
cs_old,,,,,,,,,,
CS1.1.1.1,0,0.77158466000000000000,0.18134972000000000000,0.40088127000000000000,0.02012044000000000000,0,0.08077859000000000000,0,0,1.22356738000000000000
CS1.1.1.2,0,0.85468720000000000000,0.39488423000000000000,0.00630418000000000000,0,0,0,0,0,0.04307110000000000000
CS1.1.2.1,0,1.03019264000000000000,1.00606014000000000000,2.1214444100000000,0,0,0,0,0,1.68130970000000000000
CS1.1.2.2,0,0,0,0.13656837000000000000,0,0,0,0,0,0
CS2.1.1.1,0,0.08408434000000000000,0,0.35394291000000000000,0,0,2.4830501300000000,2.8042027800000000,1.39405100000000000000,34.6319312400000000
CS2.1.2,0,0,0,0.73478079000000000000,0,0,0.67018056000000000000,0,0,10.1475997900000000
CS2.1.3,0,0.07158986000000000000,0,0.57602097000000000000,0,0,0,0,0,23.5232454800000000
CS2.2.1,0.00096177000000000000,4.9698640900000000,2.1550538900000000,12.0176101500000000,0,2.3671297900000000,15.7820810700000000,0.70128836000000000000,40.6816195400000000,12.9973178400000000


In [13]:
print(" "*10, end="|")
for col in df.columns:
    print(f"{col: >10}", end=" |")
print()
for index, row in df.iterrows():
    print(f"{index: >10}", end="|")
    for col in df.columns:
        print(f"{row[col]: >10.2f}", end=" |")
    print()

          |       nan | CS1.1.1.1 | CS1.1.1.2 | CS1.1.2.1 | CS1.1.2.2 |   CS1.2.2 | CS2.1.1.1 |   CS2.1.2 |   CS2.1.3 |   CS2.2.1 |
 CS1.1.1.1|      0.00 |      0.77 |      0.18 |      0.40 |      0.02 |      0.00 |      0.08 |      0.00 |      0.00 |      1.22 |
 CS1.1.1.2|      0.00 |      0.85 |      0.39 |      0.01 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.04 |
 CS1.1.2.1|      0.00 |      1.03 |      1.01 |      2.12 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      1.68 |
 CS1.1.2.2|      0.00 |      0.00 |      0.00 |      0.14 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |
 CS2.1.1.1|      0.00 |      0.08 |      0.00 |      0.35 |      0.00 |      0.00 |      2.48 |      2.80 |      1.39 |     34.63 |
   CS2.1.2|      0.00 |      0.00 |      0.00 |      0.73 |      0.00 |      0.00 |      0.67 |      0.00 |      0.00 |     10.15 |
   CS2.1.3|      0.00 |      0.07 |      0.00 |      0.58 |      0.00 |     

In [14]:
list(df.columns)

[nan,
 'CS1.1.1.1',
 'CS1.1.1.2',
 'CS1.1.2.1',
 'CS1.1.2.2',
 'CS1.2.2',
 'CS2.1.1.1',
 'CS2.1.2',
 'CS2.1.3',
 'CS2.2.1']

In [15]:
result = {
    index: {col: row[col] for col in df.columns} for index, row in df.iterrows()
}
result

{'CS1.1.1.1': {nan: 0,
  'CS1.1.1.1': Decimal('0.77158466000000000000'),
  'CS1.1.1.2': Decimal('0.18134972000000000000'),
  'CS1.1.2.1': Decimal('0.40088127000000000000'),
  'CS1.1.2.2': Decimal('0.02012044000000000000'),
  'CS1.2.2': 0,
  'CS2.1.1.1': Decimal('0.08077859000000000000'),
  'CS2.1.2': 0,
  'CS2.1.3': 0,
  'CS2.2.1': Decimal('1.22356738000000000000')},
 'CS1.1.1.2': {nan: 0,
  'CS1.1.1.1': Decimal('0.85468720000000000000'),
  'CS1.1.1.2': Decimal('0.39488423000000000000'),
  'CS1.1.2.1': Decimal('0.00630418000000000000'),
  'CS1.1.2.2': 0,
  'CS1.2.2': 0,
  'CS2.1.1.1': 0,
  'CS2.1.2': 0,
  'CS2.1.3': 0,
  'CS2.2.1': Decimal('0.04307110000000000000')},
 'CS1.1.2.1': {nan: 0,
  'CS1.1.1.1': Decimal('1.03019264000000000000'),
  'CS1.1.1.2': Decimal('1.00606014000000000000'),
  'CS1.1.2.1': Decimal('2.1214444100000000'),
  'CS1.1.2.2': 0,
  'CS1.2.2': 0,
  'CS2.1.1.1': 0,
  'CS2.1.2': 0,
  'CS2.1.3': 0,
  'CS2.2.1': Decimal('1.68130970000000000000')},
 'CS1.1.2.2': {nan: 0,

In [16]:
headers = [_ for _ in list(result.values())[0].keys()]
headers

[nan,
 'CS1.1.1.1',
 'CS1.1.1.2',
 'CS1.1.2.1',
 'CS1.1.2.2',
 'CS1.2.2',
 'CS2.1.1.1',
 'CS2.1.2',
 'CS2.1.3',
 'CS2.2.1']